In [1]:
import logging

import numpy

import cicada.additive
import cicada.encoder
from cicada.communicator import SocketCommunicator

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, modulus=251, precision=4)

    secret = numpy.array(3) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}", src=0)

    secret_share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())
    
    decomposition_share = protocol.bit_decompose(secret_share)
    decomposition = protocol.reveal(decomposition_share)
    log.info(f"Player {communicator.rank} decomposition shape: {decomposition.shape}", src=1)
    log.info(f"Player {communicator.rank} decomposition: {decomposition}", src=2)

SocketCommunicator.run(main, world_size=3);

INFO:cicada.communicator.socket:Comm 'world' player 1 rendezvous with tcp://127.0.0.1:57276 from tcp://127.0.0.1:57277.
INFO:cicada.communicator.socket:Comm 'world' player 0 rendezvous with tcp://127.0.0.1:57276 from tcp://127.0.0.1:57276.
INFO:cicada.communicator.socket:Comm 'world' player 2 rendezvous with tcp://127.0.0.1:57276 from tcp://127.0.0.1:57278.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 secret: 3
INFO:root:Player 1 decomposition shape: (8,)
INFO:root:Player 2 decomposition: [0 0 1 1 0 0 0 0]
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator freed.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.socket:Player 0 return: None
INFO:cicada.comm

Note that the decomposition contains 8 bits with the integer value "3" stored in the first four as $0 0 1 1$, and the fractional value ".0" stored in the last four as $0 0 0 0$.

To cement our understanding, let's run the same computation with something more complex than an integer:

In [2]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, modulus=251, precision=4)

    secret = numpy.array(3.5) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}", src=0)

    secret_share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())
    
    decomposition_share = protocol.bit_decompose(secret_share)
    decomposition = protocol.reveal(decomposition_share)
    log.info(f"Player {communicator.rank} decomposition shape: {decomposition.shape}", src=1)
    log.info(f"Player {communicator.rank} decomposition: {decomposition}", src=2)

SocketCommunicator.run(main, world_size=3);

INFO:cicada.communicator.socket:Comm 'world' player 0 rendezvous with tcp://127.0.0.1:57294 from tcp://127.0.0.1:57294.
INFO:cicada.communicator.socket:Comm 'world' player 1 rendezvous with tcp://127.0.0.1:57294 from tcp://127.0.0.1:57296.
INFO:cicada.communicator.socket:Comm 'world' player 2 rendezvous with tcp://127.0.0.1:57294 from tcp://127.0.0.1:57298.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 secret: 3.5
INFO:root:Player 1 decomposition shape: (8,)
INFO:root:Player 2 decomposition: [0 0 1 1 1 0 0 0]
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator freed.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.socket:Player 0 return: None
INFO:cicada.co

Now we see that the integer portion of our value is still stored in the first four bits as $0011$ while the fractional portion "0.5" is stored in the last four as $1000$.